In \[15\]:

    from sklearn.datasets import load_boston
    from sklearn.linear_model import LinearRegression
    import pandas as pd
    import numpy

In \[16\]:

    data = load_boston()
    X = pd.DataFrame(data.data, columns=data.feature_names).values
    y = pd.DataFrame(data.target).values
    model = LinearRegression()
    model.fit(X,y)

Out\[16\]:

    LinearRegression()

In \[17\]:

    x = pd.DataFrame(data.data, columns=data.feature_names)

In \[18\]:

    x.describe()

Out\[18\]:

|       | CRIM       | ZN         | INDUS      | CHAS       | NOX        | RM         | AGE        | DIS        | RAD        | TAX        | PTRATIO    | B          | LSTAT      |
|-------|------------|------------|------------|------------|------------|------------|------------|------------|------------|------------|------------|------------|------------|
| count | 506.000000 | 506.000000 | 506.000000 | 506.000000 | 506.000000 | 506.000000 | 506.000000 | 506.000000 | 506.000000 | 506.000000 | 506.000000 | 506.000000 | 506.000000 |
| mean  | 3.613524   | 11.363636  | 11.136779  | 0.069170   | 0.554695   | 6.284634   | 68.574901  | 3.795043   | 9.549407   | 408.237154 | 18.455534  | 356.674032 | 12.653063  |
| std   | 8.601545   | 23.322453  | 6.860353   | 0.253994   | 0.115878   | 0.702617   | 28.148861  | 2.105710   | 8.707259   | 168.537116 | 2.164946   | 91.294864  | 7.141062   |
| min   | 0.006320   | 0.000000   | 0.460000   | 0.000000   | 0.385000   | 3.561000   | 2.900000   | 1.129600   | 1.000000   | 187.000000 | 12.600000  | 0.320000   | 1.730000   |
| 25%   | 0.082045   | 0.000000   | 5.190000   | 0.000000   | 0.449000   | 5.885500   | 45.025000  | 2.100175   | 4.000000   | 279.000000 | 17.400000  | 375.377500 | 6.950000   |
| 50%   | 0.256510   | 0.000000   | 9.690000   | 0.000000   | 0.538000   | 6.208500   | 77.500000  | 3.207450   | 5.000000   | 330.000000 | 19.050000  | 391.440000 | 11.360000  |
| 75%   | 3.677083   | 12.500000  | 18.100000  | 0.000000   | 0.624000   | 6.623500   | 94.075000  | 5.188425   | 24.000000  | 666.000000 | 20.200000  | 396.225000 | 16.955000  |
| max   | 88.976200  | 100.000000 | 27.740000  | 1.000000   | 0.871000   | 8.780000   | 100.000000 | 12.126500  | 24.000000  | 711.000000 | 22.000000  | 396.900000 | 37.970000  |

In \[19\]:

    # Calculating the fitness value of each solution in the current population.
    def cal_pop_fitness(pop):
        fitness = model.predict(pop)
        return fitness


    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    def select_mating_pool(pop, fitness, num_parents):
        parents = numpy.empty((num_parents, pop.shape[1])) #50*13
        for parent_num in range(num_parents):
            max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
            max_fitness_idx = max_fitness_idx[0][0]
            parents[parent_num, :] = pop[max_fitness_idx, :]
            fitness[max_fitness_idx] = -99999999999
        return parents

    def crossover(parents, offspring_size): #50 50
        offspring = numpy.empty(offspring_size)
        # The point at which crossover takes place between two parents. Usually it is at the center.
        crossover_point = numpy.uint8(offspring_size[1]/2) #6
        for k in range(offspring_size[0]):
            # Index of the first parent to mate.
            parent1_idx = k%parents.shape[0] #selecting 1st k values
            # Index of the second parent to mate.
            parent2_idx = (k+1)%parents.shape[0]#selecting k+1 to last values
            # The new offspring will have its first half of its genes taken from the first parent.
            offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
            # The new offspring will have its second half of its genes taken from the second parent.
            offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
        return offspring

    def mutation(offspring_crossover):
        # Mutation changes a single gene in each offspring randomly.
        for idx in range(offspring_crossover.shape[0]):
            # The random value to be added to the gene.
            random_value = numpy.random.uniform(0.2, 0.9, 1)
            offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
        return offspring_crossover

In \[25\]:

    num_features = 13
    sol_per_pop = 100
    num_parents_mating = 20

    # Defining the population size.
    pop_size = (sol_per_pop,num_features) 

    #Creating the initial population.
    new_population = X[:100]

    num_generations = 5
    for generation in range(num_generations):
        print("Generation : ", generation)
        
        # Measing the fitness of each chromosome in the population.
        fitness = cal_pop_fitness(new_population)
       
        # Selecting the best parents in the population for mating.
        parents = select_mating_pool(new_population, fitness, 
                                          num_parents_mating)

        # Generating next generation using crossover.
        offspring_crossover = crossover(parents,
                                           offspring_size=(pop_size[0]-parents.shape[0], num_features))

        # Adding some variations to the offsrping using mutation.
        offspring_mutation = mutation(offspring_crossover)

        # Creating the new population based on the parents and offspring.
        new_population[0:parents.shape[0], :] = parents
        new_population[parents.shape[0]:, :] = offspring_mutation

        # The best result in the current iteration.
        print("Best result : ", model.predict(new_population))

    # Getting the best solution after iterating finishing all generations.
    #At first, the fitness is calculated for each solution in the final generation.
    fitness = cal_pop_fitness(new_population)
    # Then return the index of that solution corresponding to the best fitness.
    best_match_idx = numpy.where(fitness == numpy.max(fitness))

    print("Best solution : ", new_population[best_match_idx, :])
    print("Best solution fitness : ", fitness[best_match_idx])

    Generation :  0
    Best result :  [[44.67308339]
     [43.30239043]
     [43.18949844]
     [42.03193124]
     [41.84728168]
     [40.98928501]
     [40.84555721]
     [40.8395575 ]
     [40.55765844]
     [40.33963075]
     [39.81461867]
     [39.31792393]
     [38.79756966]
     [38.37102453]
     [38.0481048 ]
     [37.58605755]
     [37.51109239]
     [37.4944739 ]
     [37.20347455]
     [37.16631331]
     [39.28297601]
     [28.16794755]
     [37.9074842 ]
     [39.82472654]
     [34.92193955]
     [25.84310343]
     [33.20726289]
     [28.31311144]
     [29.9626808 ]
     [24.59229377]
     [31.73995189]
     [30.83032141]
     [31.28812692]
     [29.5056775 ]
     [24.90745005]
     [23.78237819]
     [29.81339369]
     [25.04471651]
     [23.52191937]
     [29.10300031]
     [37.52785564]
     [27.06593952]
     [34.67552049]
     [39.33946912]
     [33.77520835]
     [24.28491306]
     [32.13797186]
     [32.42824668]
     [33.44547755]
     [34.9565456 ]
     [35.1829534 ]
     [26.12031095]
     [34.70776759]
     [25.27219652]
     [25.30629118]
     [24.93475983]
     [24.86957417]
     [36.51106509]
     [22.90808228]
     [22.8222323 ]
     [46.97238682]
     [31.46062671]
     [38.37294652]
     [32.21368163]
     [35.35748554]
     [24.86874872]
     [37.75910245]
     [27.19920031]
     [26.14602391]
     [25.56636347]
     [40.43278787]
     [28.57537129]
     [23.55608929]
     [30.47837457]
     [31.97243317]
     [27.01228125]
     [32.28011201]
     [27.31776679]
     [26.9217619 ]
     [21.1223757 ]
     [39.32279617]
     [33.54322798]
     [27.27041127]
     [34.60931954]
     [29.68537871]
     [20.46540899]
     [39.54397721]
     [28.32203529]
     [28.43525685]
     [36.73661967]
     [28.52305811]
     [30.86941844]
     [29.79117813]
     [23.84409904]
     [23.91200812]
     [27.0736942 ]
     [29.51777958]
     [33.3335412 ]
     [30.28673872]
     [23.06580319]]
    Generation :  1
    Best result :  [[46.97238682]
     [44.67308339]
     [43.30239043]
     [43.18949844]
     [42.03193124]
     [41.84728168]
     [40.98928501]
     [40.84555721]
     [40.8395575 ]
     [40.55765844]
     [40.43278787]
     [40.33963075]
     [39.82472654]
     [39.81461867]
     [39.54397721]
     [39.33946912]
     [39.32279617]
     [39.31792393]
     [39.28297601]
     [38.79756966]
     [32.09752309]
     [40.75017316]
     [28.81500608]
     [29.62971434]
     [36.23125407]
     [33.1639577 ]
     [29.601074  ]
     [36.26152133]
     [27.6954897 ]
     [35.94714989]
     [23.56823539]
     [30.97851812]
     [24.7692649 ]
     [36.37126662]
     [25.51921777]
     [33.35766645]
     [30.4118424 ]
     [27.77204531]
     [22.89547142]
     [32.55929653]
     [35.74867049]
     [36.76751948]
     [33.32633354]
     [36.16983421]
     [28.78327946]
     [38.5930689 ]
     [25.22129135]
     [32.35549744]
     [26.67594909]
     [33.47552223]
     [29.0578547 ]
     [36.0033613 ]
     [26.89372578]
     [30.45725673]
     [24.76201694]
     [34.59888066]
     [31.40771133]
     [25.23534072]
     [20.83429135]
     [29.93105558]
     [26.32950624]
     [40.72987302]
     [27.76902294]
     [31.98629141]
     [37.14034859]
     [29.62683114]
     [20.76791041]
     [41.15958164]
     [32.04585675]
     [33.0793604 ]
     [22.82043278]
     [35.25370761]
     [31.66336158]
     [32.40185512]
     [25.53316889]
     [29.25760968]
     [30.94446728]
     [28.51778385]
     [20.93123483]
     [28.9572939 ]
     [33.07607497]
     [45.07319704]
     [25.76922774]
     [30.65839006]
     [40.72319716]
     [40.01672589]
     [23.67144805]
     [32.8670589 ]
     [36.63871863]
     [26.73689963]
     [22.85612827]
     [34.01747453]
     [22.7547813 ]
     [29.59340977]
     [29.2146937 ]
     [30.76103494]
     [23.39938103]
     [33.15229064]
     [21.82113417]
     [32.65656357]]
    Generation :  2
    Best result :  [[46.97238682]
     [45.07319704]
     [44.67308339]
     [43.30239043]
     [43.18949844]
     [42.03193124]
     [41.84728168]
     [41.15958164]
     [40.98928501]
     [40.84555721]
     [40.8395575 ]
     [40.75017316]
     [40.72987302]
     [40.72319716]
     [40.55765844]
     [40.43278787]
     [40.33963075]
     [40.01672589]
     [39.82472654]
     [39.81461867]
     [31.19953386]
     [27.85435949]
     [42.00334974]
     [22.1509774 ]
     [30.52762918]
     [35.46721265]
     [28.75170436]
     [30.2117051 ]
     [25.97024764]
     [37.49488796]
     [32.43925792]
     [35.17937609]
     [23.50398802]
     [25.84362336]
     [38.31295588]
     [21.05972049]
     [37.66596054]
     [24.84973446]
     [25.67624566]
     [40.08941061]
     [33.8070677 ]
     [33.48238807]
     [35.62326546]
     [23.90244486]
     [37.61479402]
     [35.4549057 ]
     [38.52984573]
     [31.7909402 ]
     [27.53008873]
     [38.28626885]
     [33.16448263]
     [28.98467081]
     [28.03932237]
     [37.75293469]
     [31.98266197]
     [31.76499316]
     [40.63214887]
     [23.61734088]
     [31.52216598]
     [36.89189053]
     [38.6842843 ]
     [24.23627868]
     [40.84997809]
     [22.53915718]
     [32.91200718]
     [29.74454525]
     [27.76124574]
     [32.11341127]
     [28.03192791]
     [35.4768244 ]
     [38.65998279]
     [26.60517969]
     [28.75441786]
     [29.15939052]
     [32.89258429]
     [28.49302951]
     [35.65756413]
     [31.16248356]
     [29.15603089]
     [33.88033156]
     [39.95556708]
     [34.21713513]
     [44.73826048]
     [30.92448743]
     [37.25792505]
     [33.90653354]
     [29.40209699]
     [30.0578729 ]
     [32.0867432 ]
     [34.32696481]
     [36.81370393]
     [32.77854179]
     [25.92035082]
     [32.79759451]
     [34.50845872]
     [30.01346912]
     [28.83533329]
     [28.23467552]
     [30.93045697]
     [39.70786412]]
    Generation :  3
    Best result :  [[46.97238682]
     [45.07319704]
     [44.73826048]
     [44.67308339]
     [43.30239043]
     [43.18949844]
     [42.03193124]
     [42.00334974]
     [41.84728168]
     [41.15958164]
     [40.98928501]
     [40.84997809]
     [40.84555721]
     [40.8395575 ]
     [40.75017316]
     [40.72987302]
     [40.72319716]
     [40.63214887]
     [40.55765844]
     [40.43278787]
     [35.23143396]
     [36.5464867 ]
     [31.80750074]
     [34.76904567]
     [23.14048506]
     [32.80597519]
     [36.6079804 ]
     [27.25317497]
     [35.02843028]
     [36.85585268]
     [26.85600308]
     [18.79658124]
     [32.4487099 ]
     [31.74489983]
     [36.34366168]
     [33.97393896]
     [38.64619896]
     [35.41851255]
     [27.58072178]
     [31.3810818 ]
     [42.21389573]
     [31.67012999]
     [32.31442922]
     [38.66049904]
     [24.33366344]
     [33.50872704]
     [42.29876499]
     [32.74992181]
     [37.4478359 ]
     [30.2981291 ]
     [31.65705302]
     [22.30306701]
     [37.33388534]
     [38.85941948]
     [26.63608533]
     [33.42893761]
     [38.53579287]
     [27.6754675 ]
     [38.15287239]
     [37.07378686]
     [37.40519526]
     [37.60884644]
     [29.94300864]
     [35.94002697]
     [25.12207064]
     [31.49286762]
     [40.96063776]
     [27.7260325 ]
     [30.94390418]
     [30.14892289]
     [35.96616892]
     [20.02295703]
     [32.87622998]
     [39.03869466]
     [25.45782121]
     [32.63091071]
     [36.54193943]
     [26.85999852]
     [31.57552319]
     [34.851573  ]
     [31.6196982 ]
     [33.37761695]
     [31.85193452]
     [44.91794052]
     [24.02357513]
     [32.91259257]
     [39.75925935]
     [30.60373464]
     [31.22961894]
     [28.40966013]
     [27.59918889]
     [21.49278365]
     [40.75496393]
     [35.28716651]
     [27.99849136]
     [29.41148776]
     [33.69587356]
     [27.553782  ]
     [34.18630599]
     [27.70869118]]
    Generation :  4
    Best result :  [[46.97238682]
     [45.07319704]
     [44.91794052]
     [44.73826048]
     [44.67308339]
     [43.30239043]
     [43.18949844]
     [42.29876499]
     [42.21389573]
     [42.03193124]
     [42.00334974]
     [41.84728168]
     [41.15958164]
     [40.98928501]
     [40.96063776]
     [40.84997809]
     [40.84555721]
     [40.8395575 ]
     [40.75496393]
     [40.75017316]
     [43.20881089]
     [30.39590151]
     [33.9916202 ]
     [24.08423333]
     [38.49424985]
     [28.40939523]
     [37.84230102]
     [35.65297229]
     [22.33251325]
     [34.95463767]
     [35.33151052]
     [34.23499745]
     [27.52902703]
     [31.30958645]
     [33.21881169]
     [19.56064756]
     [34.73520279]
     [33.04457584]
     [38.36425881]
     [34.11611264]
     [35.4074447 ]
     [37.37607279]
     [39.44985051]
     [26.72877521]
     [39.08569002]
     [29.34021163]
     [41.2225401 ]
     [30.40389745]
     [24.83314982]
     [39.92349335]
     [27.53426111]
     [34.92776383]
     [34.06158786]
     [29.00757607]
     [35.46404104]
     [22.25071573]
     [32.9819389 ]
     [32.24529492]
     [36.99042353]
     [28.46175905]
     [37.65852942]
     [33.66649031]
     [36.15338088]
     [31.42651732]
     [34.77593073]
     [34.23021783]
     [44.09836842]
     [27.07608289]
     [30.06037073]
     [32.41526215]
     [29.2253022 ]
     [31.21684688]
     [27.28154173]
     [30.58736431]
     [37.01200542]
     [23.87138267]
     [36.74032982]
     [34.58621584]
     [38.87858492]
     [28.2265943 ]
     [31.96440325]
     [38.93781486]
     [39.95461749]
     [32.75699377]
     [38.60698665]
     [31.74137669]
     [37.33082249]
     [32.39539495]
     [30.75566947]
     [34.9123715 ]
     [26.5008983 ]
     [32.12446326]
     [33.3847295 ]
     [31.60198466]
     [34.00490367]
     [22.98659092]
     [36.1866432 ]
     [33.73847286]
     [38.40024492]
     [27.47181017]]
    Best solution :  [[[1.50100000e-02 9.00000000e+01 1.21000000e+00 1.00000000e+00
       6.07507663e-01 7.92300000e+00 8.69000000e+01 1.80100000e+00
       5.00000000e+00 2.64000000e+02 1.30000000e+01 3.89700000e+02
       5.12000000e+00]]

     [[1.50100000e-02 9.00000000e+01 1.21000000e+00 1.00000000e+00
       6.07507663e-01 7.92300000e+00 8.69000000e+01 1.80100000e+00
       5.00000000e+00 2.64000000e+02 1.30000000e+01 3.89700000e+02
       5.12000000e+00]]]
    Best solution fitness :  [46.97238682]

In \[26\]:

    new_population[best_match_idx, :]

Out\[26\]:

    array([[[1.50100000e-02, 9.00000000e+01, 1.21000000e+00, 1.00000000e+00,
             6.07507663e-01, 7.92300000e+00, 8.69000000e+01, 1.80100000e+00,
             5.00000000e+00, 2.64000000e+02, 1.30000000e+01, 3.89700000e+02,
             5.12000000e+00]],

           [[1.50100000e-02, 9.00000000e+01, 1.21000000e+00, 1.00000000e+00,
             6.07507663e-01, 7.92300000e+00, 8.69000000e+01, 1.80100000e+00,
             5.00000000e+00, 2.64000000e+02, 1.30000000e+01, 3.89700000e+02,
             5.12000000e+00]]])

In \[27\]:

    model.predict(new_population[0].reshape(-1,13))

Out\[27\]:

    array([[46.97238682]])

In \[ \]: